In [6]:
"""
Slimmed-down BB84 protocol simulation using Amazon Braket LocalSimulator.
Runs 100% locally (no AWS account or billing required).
"""

import random
from braket.circuits import Circuit
from braket.devices import LocalSimulator

# Parameters
NUM_QUBITS = 6      # qubits per round
ROUNDS = 50         # number of rounds
SHOTS = 1           # shots per circuit (we only need 1 sample per round)

# Helper: Alice's qubit preparation
def prepare_alice_ops(circ, q, bit, basis):
    if basis == "Z":
        if bit == 1:
            circ.x(q)
    elif basis == "X":
        if bit == 1:
            circ.x(q)
        circ.h(q)
    else:
        raise ValueError("basis must be 'Z' or 'X'")

# Helper: Bob’s measurement prep
def apply_bob_measurement_prep(circ, q, basis):
    if basis == "X":
        circ.h(q)

# Build one BB84 circuit
def build_bb84_circuit(alice_bits, alice_bases, bob_bases):
    circ = Circuit()
    for q in range(NUM_QUBITS):
        prepare_alice_ops(circ, q, alice_bits[q], alice_bases[q])
    for q in range(NUM_QUBITS):
        apply_bob_measurement_prep(circ, q, bob_bases[q])
    for q in range(NUM_QUBITS):
        circ.measure(q)
    return circ

# Sifting
def sift(alice_bases_list, bob_bases_list, alice_bits_list, bob_bits_list):
    sifted_a, sifted_b = [], []
    for a_bases, b_bases, a_bits, b_bits in zip(
        alice_bases_list, bob_bases_list, alice_bits_list, bob_bits_list
    ):
        for i in range(NUM_QUBITS):
            if a_bases[i] == b_bases[i]:
                sifted_a.append(str(a_bits[i]))
                sifted_b.append(str(b_bits[i]))
    return sifted_a, sifted_b

def main():
    device = LocalSimulator("braket_dm")
    alice_bits_list, alice_bases_list, bob_bases_list, bob_bits_list = [], [], [], []

    # Generate rounds
    for r in range(ROUNDS):
        alice_bits = [random.randint(0, 1) for _ in range(NUM_QUBITS)]
        alice_bases = [random.choice(["Z", "X"]) for _ in range(NUM_QUBITS)]
        bob_bases = [random.choice(["Z", "X"]) for _ in range(NUM_QUBITS)]

        circ = build_bb84_circuit(alice_bits, alice_bases, bob_bases)
        result = device.run(circ, shots=SHOTS).result()
        counts = result.measurement_counts
        outcome = max(counts, key=counts.get)  # most frequent outcome
        bob_bits = [int(b) for b in outcome[::-1]]  # reverse bit order for consistency

        alice_bits_list.append(alice_bits)
        alice_bases_list.append(alice_bases)
        bob_bases_list.append(bob_bases)
        bob_bits_list.append(bob_bits)

    # Sifting
    alice_sifted, bob_sifted = sift(
        alice_bases_list, bob_bases_list, alice_bits_list, bob_bits_list
    )

    print("Sifted key length:", len(alice_sifted))
    print("Alice key (first 32 bits):", "".join(alice_sifted[:32]))
    print("Bob   key (first 32 bits):", "".join(bob_sifted[:32]))

    # Error rate (QBER)
    if alice_sifted:
        errors = sum(a != b for a, b in zip(alice_sifted, bob_sifted))
        qber = errors / len(alice_sifted)
        print(f"Estimated QBER: {qber:.3f} ({errors}/{len(alice_sifted)})")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.12/dist-packages/braket/default_simulator/simulator.py:333: UserWarning: You are running a noise-free circuit on the density matrix simulator. Consider running this circuit on the state vector simulator: LocalSimulator("default") for a better user experience.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/braket/default_simulator/simulator.py:333: UserWarning: You are running a noise-free circuit on the density matrix simulator. Consider running this circuit on the state vector simulator: LocalSimulator("default") for a better user experience.
  warnings.warn(


Sifted key length: 154
Alice key (first 32 bits): 01010011001000001101011101000000
Bob   key (first 32 bits): 00100100010100101100010110001000
Estimated QBER: 0.519 (80/154)


In [2]:
!pip install amazon-braket-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.4/370.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.8/537.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.9.3
    Uninstalling antlr4-python3-runtime-4.9.3:
      Successfully uninstalled antlr4-py